In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

# Read
df = pd.read_csv('Data/cleaned_reviews_with_sentiment.csv')

# Total Reviews
print(f"> Total Reviews: {df.shape[0]:,}")

# View
display(df.head())

> Total Reviews: 100


,reviews_text,sentiment
0,the hotel was great. staff went above and beyond bringing hot tea to room at 11:30 pm at no charge!,POSITIVE
1,"a wonderful hotel - would definitely stay there whenever we are in alexandria again. wonderful staff. we remarked that the first room was very far from the elevator, and they immediately moved us to a much nicer/larger room across from the elevator without question. i hope i am able to stay at a kimpton hotel on future trips.",POSITIVE
2,"i needed a place to stay in va that was close to or in alexandria and the monaco was available for a great value. i enjoyed my stay, room was comfortable and clean. was disappointed on parking cost but i was able to work around that. very convenient to old town.",POSITIVE
3,"the hotel itself was very nice. the desk staff were very attentive and helpful. the rooms were nice but not dusted/cleaned adequately each day. the problems we had were few but definitely impacted whether we would stay there again. the hallways and elevators were not air-conditioned and were in the high 80's in july. also, with no complimentary breakfast offered, the free coffee was in high demand. the coffee hours ended at 9:00 a.m. and the coffee was always empty, missing ""to go"" cups, or out of cream and sugar. a daily conversation with the desk staff on this issue grew tiresome with our 7 day stay. look elsewhere if service is important to you.",NEGATIVE
4,"we arrive around 1 am, for a 5 night stay, and the lady could not find our reservation. after a while she she found it but could not tell us a thing about our expedia gold benefits. she told us it was not her problem a we should ask the manager eventually.",NEGATIVE


#### MODEL

In [3]:
from langchain.chat_models import init_chat_model

# MODEL
model = init_chat_model(model="claude-haiku-4-5-20251001", temperature=0)

#### PYDANTIC MODEL

In [5]:
# Import BaseModel to construct the primary data structure & Field to attach LLM instructions
from pydantic import BaseModel, Field

# Define the extraction class, inheriting from BaseModel
class ReviewInsight(BaseModel):
    
    why: str = Field(
        description="A concise one-sentence (15 words max) explanation of why this sentiment was chosen.")

#### PYDANTIC PARSER

In [6]:
from langchain_core.output_parsers import PydanticOutputParser

# Initialise OUTPUT parser with defined schema
parser = PydanticOutputParser(pydantic_object=ReviewInsight)

#### PROMPT

In [8]:
from langchain_core.prompts import ChatPromptTemplate

# PROMPT
prompt = ChatPromptTemplate.from_template(
    """
    You are an expert hospitality data analyst.
    
    The review sentiment is {sentiment}.
    
    Analyse the following hotel review and extract insights based on the provided rules.
    
    Review: {review}
    
    {format_instructions}
    """
)

# INJECT PYDANTIC PARSER RULES INTO PROMPT
prompt = prompt.partial(format_instructions=parser.get_format_instructions())

#### CREATE THE CHAIN

In [9]:
# Prompt -> LLM -> Parser
chain = prompt | model | parser

#### EXECUTION FUNCTION

In [11]:
def process_review(row):
    
    try:
        
        # Pass both review text & sentiment label into the chain
        result = chain.invoke({"review": row['reviews_text'], 
                               "sentiment": row['sentiment']})
        
        return pd.Series([result.why])
    
    except Exception:
        return pd.Series(["ERROR"])

In [13]:
# Apply
df['why'] = df.apply(process_review, axis=1)

In [14]:
# View
display(df.head())

,reviews_text,sentiment,why
0,the hotel was great. staff went above and beyond bringing hot tea to room at 11:30 pm at no charge!,POSITIVE,Staff provided exceptional service by delivering complimentary hot tea late night.
1,"a wonderful hotel - would definitely stay there whenever we are in alexandria again. wonderful staff. we remarked that the first room was very far from the elevator, and they immediately moved us to a much nicer/larger room across from the elevator without question. i hope i am able to stay at a kimpton hotel on future trips.",POSITIVE,"Excellent staff service, room upgrade, and overall wonderful experience."
2,"i needed a place to stay in va that was close to or in alexandria and the monaco was available for a great value. i enjoyed my stay, room was comfortable and clean. was disappointed on parking cost but i was able to work around that. very convenient to old town.",POSITIVE,"Guest appreciated value, comfort, cleanliness, and location despite parking concerns."
3,"the hotel itself was very nice. the desk staff were very attentive and helpful. the rooms were nice but not dusted/cleaned adequately each day. the problems we had were few but definitely impacted whether we would stay there again. the hallways and elevators were not air-conditioned and were in the high 80's in july. also, with no complimentary breakfast offered, the free coffee was in high demand. the coffee hours ended at 9:00 a.m. and the coffee was always empty, missing ""to go"" cups, or out of cream and sugar. a daily conversation with the desk staff on this issue grew tiresome with our 7 day stay. look elsewhere if service is important to you.",NEGATIVE,"Multiple service failures including poor housekeeping, inadequate climate control, and unreliable complimentary coffee."
4,"we arrive around 1 am, for a 5 night stay, and the lady could not find our reservation. after a while she she found it but could not tell us a thing about our expedia gold benefits. she told us it was not her problem a we should ask the manager eventually.",NEGATIVE,Staff unable to locate reservation and dismissive about guest benefits.


#### SAVE

In [15]:
# Save for LangChain ingestion
df.to_csv('Data/cleaned_reviews_actionable.csv', index=False)